In [ ]:
# Generate Portfolios based on market cap pickinig the top N stocks

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [2]:
# Get the CRSP Data
# Derived from readcrsp.ipynb
crsp_m = pd.read_csv('data/crspmsf.csv', parse_dates=[0])
crsp_m['permno'] = crsp_m['permno'].astype(int)
crsp_m['mktcap'] = abs(crsp_m['shrout']*crsp_m['altprc'])
returndata = crsp_m[['date','permno','ret','mktcap']]

In [3]:
# Remove the stocks where we dont have returns data
returndata = returndata[~returndata.ret.isna()]

In [4]:
# Sort the data date wise
returndata = returndata.sort_values(['date', 'permno'])

In [5]:
# Update the date to retain ony the year and month string
returndata['date'] = returndata['date'].apply(lambda x: x.replace('-', '')[:6])

In [6]:
returndata.head()

,date,permno,ret,mktcap
920,192601,10006,0.032732,66150.00
3791,192601,10022,0.017857,11400.00
5647,192601,10030,0.161667,27183.00
8872,192601,10049,0.141892,20875.00
9913,192601,10057,-0.035714,5906.25


In [7]:
# Get the Fama-French Factors (for Risk Free Rate)
ff_factors = pd.read_csv('data/F-F_Research_Data_Factors-2.csv', skiprows=4, names='date,mkt_rf,smb,hml,rf'.split(','))
ff_factors = ff_factors[ff_factors.date.str.len() == 6]

In [8]:
# Join the ff factors and the CRSP returns
returndata = pd.merge(returndata, ff_factors[['date', 'rf']], how='left', on=['date'])

In [9]:
# Get the excess returns
returndata['excess_ret'] = returndata['ret'] - returndata['rf'].astype(float)

In [14]:
# Create a portfolio and get the portfolio excess return
def get_portfolio_excess_ret(data_dt, num_stocks):
    data_dt_temp = data_dt.copy()
    data_dt_temp = data_dt_temp.sort_values('mktcap', ascending=False)[:num_stocks]
    total_mkt_cap = np.sum(data_dt_temp.mktcap)
    return np.sum(data_dt_temp.excess_ret*data_dt_temp.mktcap)/total_mkt_cap

In [15]:
# Number of stocks for the Benchmark
N = 500

In [21]:
portfolio_excess_ret = returndata.groupby('date').apply(lambda x: get_portfolio_excess_ret(x, N))
returndata = pd.merge(returndata, portfolio_excess_ret.reset_index().rename(columns={0: 'benchmark_excess_ret'}), on='date', how='left')

In [22]:
returndata

,date,permno,ret,mktcap,rf,excess_ret,benchmark_excess_ret_x,benchmark_excess_ret_y
0,192601,10006,0.032732,6.615000e+04,NaN,NaN,0.000000,0.000000
1,192601,10022,0.017857,1.140000e+04,NaN,NaN,0.000000,0.000000
2,192601,10030,0.161667,2.718300e+04,NaN,NaN,0.000000,0.000000
3,192601,10049,0.141892,2.087500e+04,NaN,NaN,0.000000,0.000000
4,192601,10057,-0.035714,5.906250e+03,NaN,NaN,0.000000,0.000000
...,...,...,...,...,...,...,...,...
4652772,202203,93426,0.023559,4.054758e+05,0.01,0.013559,0.031048,0.031048
4652773,202203,93427,0.049935,3.889284e+06,0.01,0.039935,0.031048,0.031048
4652774,202203,93429,-0.024469,1.219740e+07,0.01,-0.034469,0.031048,0.031048
4652775,202203,93434,-0.073913,8.287404e+04,0.01,-0.083913,0.031048,0.031048


In [26]:
portfolio_excess_ret = portfolio_excess_ret.reset_index().rename(columns={0: 'benchmark_excess_ret'})

In [27]:
portfolio_excess_ret.to_csv('data/portfolio_excess_ret.csv')